#  Tradução de Textos - Implantação
## Utilização do modelo [MarianMT](https://huggingface.co/transformers/model_doc/marian.html) para tradução. 


* Neste exempo a tradução é feito do inglês para o português, mas ela pode ser feita em qualquer uma das línguas suportadas pelo MarianMT. 
* Para adaptar para traduções em outras línguas é necessário verificacar se há o modelo pré treinado disponível no MarianMT e adaptar o truncamento de strings do [spacy](https://spacy.io/usage/models) para o idioma desejado
* A métrica computada é o [sacrebleu](https://https://github.com/mjpost/sacrebleu) 

### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [1]:
%%writefile Model.py
import joblib
import pandas as pd
import numpy as np
from marianmt_model import MarianMTTranslator


class Model:
    
    def __init__(self):
        
        artifacts = joblib.load("/tmp/data/translate.joblib")
        self.model = artifacts["model"]
        self.all_input_columns = artifacts["all_input_columns"]
        self.input_column_name = artifacts["input_column_name"]
        self.reference_column_name = artifacts["reference_column_name"]
        self.output_column_name = artifacts["output_column_name"]
      
    def predict(self, X, feature_names, meta=None):
        
        if not feature_names:
            feature_names = self.all_input_columns
        
        df = pd.DataFrame(X,columns=feature_names)
        df_result = self.model.get_result_dataframe(df_input = df,
                                                input_column_name=self.input_column_name,
                                                output_column_name=self.output_column_name,
                                                reference_column_name=self.reference_column_name)
        return df_result
        #return df_result.to_numpy()

Overwriting Model.py


In [4]:
# import pandas as pd
# df = pd.read_csv("/tmp/data/fabc-step2-2.csv")
# X = df.to_numpy()

In [5]:
# from Model import Model
# model = Model()
# result = model.predict(X,None)
# result